The input to this notebook is from CleanUpMetadata_NoVioUse.ipynb

# IMPUTATION METHODS
After reading alot about imputation methods (search item nonresponse on Google), it is clear to me we need to run some type of algorithm to help fill in all of the missing data.

https://medium.com/ibm-data-science-experience/missing-data-conundrum-exploration-and-imputation-techniques-9f40abe0fd87
At the bottom described what I was thinking, using machine learning to find the best values...
"Machine learning algorithms like eXtreme Gradient Boosting (xgboost) automatically learn the best imputation value for the missing data based on the training loss reduction."

I think trying MICE with fancyimpute as well as xgboost research may be a great approach 

https://github.com/iskandr/fancyimpute

https://xgboost.readthedocs.io/en/latest/

# TEST IMPUTATION
At first, to test out imputation methods, we need to gather a sample of the data that has NO missing values, first we need to clean up the discrete and boolean values, and then grab the no missings.  THEN we need to randomly select throughout the matrix a proportional amount of NaN replacement, so we can then test the mean squared error of the imputation methods.

In [24]:
import pandas as pd
import numpy as np

In [25]:
agpDF=pd.read_csv("cleanedUpMetadata_noVio_AGP_humfece.csv",low_memory=False)

In [26]:
#Clearly, there should probably be some code here to avoid the same subject hosts, and just set the first imputation run, or least
#amount of error on it, into the rest of the rows with that subject.
#TODO, something to come back and do for better analysis

In [27]:
agpDF=agpDF.set_index(['sample_name','host_subject_id'])
agpDF

,,acid_reflux,acne_medication,acne_medication_otc,add_adhd,age_corrected,alcohol_consumption,alcohol_frequency,alcohol_types_beercider,alcohol_types_red_wine,alcohol_types_sour_beers,...,thyroid,tonsils_removed,types_of_plants,vegetable_frequency,vitamin_b_supplement_frequency,vitamin_d_supplement_frequency,vivid_dreams,weight_kg,whole_eggs,whole_grain_frequency
sample_name,host_subject_id,,,,,,,,,,,,,,,,,,,,,
10317.000001000,30de6efc54cd37cdfc5980b23726d3d5cc11a1d333f1ad1839c413d182e3d2e30307b560a3f314bce096a7f08eb48d20581a30eb6f7ab4fa7c7f2c745fac82f7,NaN,0.0,0.0,1.0,64.0,1.0,4.0,1.0,0.0,0.0,...,0.0,NaN,NaN,4.0,0.0,3.0,NaN,52.0,0.0,2.0
10317.000001001,d212da7fd7d7d16df1954534222cc5422868f35755406fc64a6bd01cf03c8d5bafcd1734ba70d78a21b45592a1adcc40ffe7fcfc4bc48f6da5cb46d4b21a3144,NaN,0.0,0.0,NaN,53.0,1.0,1.0,0.0,0.0,0.0,...,NaN,0.0,3.0,NaN,NaN,NaN,NaN,110.0,NaN,NaN
10317.000001002,7a1742c847e1b731ed351c6cb18fddfe1ab0db81516394f2e57f977ad55ce68d2099284489c1751114e31741276e2d7ca5d5706ede614a12225be9a9e568521f,NaN,0.0,0.0,NaN,53.0,1.0,3.0,0.0,0.0,0.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,56.0,NaN,NaN
10317.000001004,ccf34b5e1ad5cff086b39d02055c3d8d4fb3d94b518d143facf90fb63b925a18bb5e350935946aa470812ce90764c2d26e70c3a04a0beaa13ba85ab37d3e7c37,NaN,0.0,0.0,NaN,44.0,1.0,1.0,0.0,0.0,0.0,...,NaN,0.0,2.0,NaN,NaN,NaN,NaN,86.0,NaN,NaN
10317.000001008,fbbfe621d1126b916a316332c723c2841b3feba451062758fd0e9c147fd032a53d200656e499784c99360fe509b09d41fee8237956facc0d0687642ec1b8be76,NaN,0.0,0.0,NaN,66.0,0.0,0.0,0.0,0.0,0.0,...,NaN,1.0,2.0,NaN,NaN,NaN,NaN,74.0,NaN,NaN
10317.000001011,443e7134bdf8eb44466113814e754c742399c3517e61db720deb38e54432163e1a6260d079f1754b3dab3e00d1be57d31bfa6dfc11cc8d711d5287516fc4f710,NaN,0.0,0.0,NaN,64.0,1.0,4.0,0.0,0.0,0.0,...,NaN,1.0,0.0,NaN,NaN,NaN,NaN,74.0,NaN,NaN
10317.000001018,b982d3225fc14ddb745067cac81d97e0a75c66252483b092a7f8c4c629d5081c7466a7baea8ba97fbfd48527c718c4e9eb08109fca3ae083928a748de5357f9a,NaN,0.0,1.0,NaN,28.0,1.0,1.0,0.0,0.0,0.0,...,NaN,0.0,2.0,NaN,NaN,NaN,NaN,77.0,NaN,NaN
10317.000001020,17a735904e3bc72b73ef6d8ae1e6d7d571bfe2fe3d9c5c5ac91fba87e160e38c9f18b92be79fb0f48adb91516503eaea67770fb2fcb7929f8bba8940ca49f6e4,NaN,0.0,0.0,NaN,29.0,1.0,3.0,0.0,0.0,0.0,...,NaN,0.0,2.0,NaN,NaN,NaN,NaN,79.0,NaN,NaN
10317.000001022,3e843f31350e6905c2813fa3edc0ece289066e5baab22598d48a8aef9fe4081692bd9b55995d3a5151233422d0b8f50ccaf93626c1113a02c362fad786e86a92,NaN,0.0,0.0,NaN,42.0,1.0,1.0,0.0,0.0,0.0,...,NaN,0.0,1.0,NaN,NaN,NaN,NaN,91.0,NaN,NaN


In [28]:
#Lets find the percentage of nans in each columns of the overall data, to then run that against our 'pure' sample set to
#do some initial testing

#Find pure
count=0
sumArryForNans=agpDF.isnull().sum(axis=1)
minsumArryForNans = sumArryForNans[sumArryForNans == 3]

#We have 47 samples that only have 3 missing values.  Lets start with those.  Believe there are 38 unique people in it.
minsumArryForNans.count()

47

In [29]:
#Calculate percentage nans per column
def get_percentage_missing(series):
    """ Calculates percentage of NaN values in DataFrame
    :param series: Pandas DataFrame object
    :return: float
    """
    num = series.isnull().sum()
    den = len(series)
    return round(num/den, 2)

percentNansTotal=get_percentage_missing(agpDF)
percentNansTotal

acid_reflux                                                        0.46
acne_medication                                                    0.03
acne_medication_otc                                                0.04
add_adhd                                                           0.26
age_corrected                                                      0.07
alcohol_consumption                                                0.03
alcohol_frequency                                                  0.03
alcohol_types_beercider                                            0.00
alcohol_types_red_wine                                             0.00
alcohol_types_sour_beers                                           0.00
alcohol_types_spiritshard_alcohol                                  0.00
alcohol_types_unspecified                                          0.00
alcohol_types_white_wine                                           0.00
allergic_to_i_have_no_food_allergies_that_i_know_of             

In [30]:
#Grab that pure DF from above
pureDF=agpDF[agpDF.isnull().sum(axis=1)==3]
#lets remove the nan'ed columns
pureDF=pureDF.drop(['depression_bipolar_schizophrenia','food_special_unspecified','lowgrain_diet_type'],axis=1)

In [31]:
#Lets NAN-ify the pure dataframe to run against, see below
import random

lengthPure=len(pureDF)
readjustedPureNanCols=round(percentNansTotal*lengthPure)
readjustedPureNanCols

nanifiedpureDF=pureDF.copy(deep=True)

#Save missing mask
missingmask=[]

for i, j in readjustedPureNanCols.items():
    curRandosArr=[]
    for rrs in range(0,int(j)):
        curRandosArr.append(random.randint(0,46))
    for curI in curRandosArr:
        nanifiedpureDF.iloc[curI][int(j)]=np.nan
    missingmask.append(curRandosArr)

nanifiedpureDF

#Find indexes of dropped cols
#pureDF['depression_bipolar_schizophrenia','food_special_unspecified','lowgrain_diet_type']
#agpDF.columns.get_loc('lowgrain_diet_type')

#Drop the same cols we dropped before
del missingmask[42]
del missingmask[57]
del missingmask[74]

,,acid_reflux,acne_medication,acne_medication_otc,add_adhd,age_corrected,alcohol_consumption,alcohol_frequency,alcohol_types_beercider,alcohol_types_red_wine,alcohol_types_sour_beers,...,thyroid,tonsils_removed,types_of_plants,vegetable_frequency,vitamin_b_supplement_frequency,vitamin_d_supplement_frequency,vivid_dreams,weight_kg,whole_eggs,whole_grain_frequency
sample_name,host_subject_id,,,,,,,,,,,,,,,,,,,,,
10317.000039633,65547100856692be9ef5c27b03fb92409eaf98e317f84bbf69f7b3db4a65e7c14da32bb4cdba1e56c67af4abd96f7c0a5799a20b05b59838c1a5dd060d24c2fa,0.0,NaN,NaN,0.0,58.0,1.0,3.0,1.0,NaN,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,72.0,0.0,4.0
10317.000039731,2983e6ca9e4039d71c48585b8cc237f5f5143944260cb5d915a882922e450dd111971305392dfaf8c20bf75094ef4e70665ef37a2f7d44d8aa37b5e4ea6c4d11,1.0,NaN,NaN,NaN,64.0,1.0,1.0,1.0,NaN,0.0,...,0.0,1.0,1.0,3.0,4.0,4.0,1.0,70.0,3.0,3.0
10317.000039866,7e4ad3f6d351b74239d2e9388b56ec309db43f676ca8a7569d4b5dfc3278b9584ecd224f4f276b76d2efcee2052d2c3e6dc85e2655aa607232fda53b1e93de36,0.0,0.0,NaN,0.0,55.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,4.0,1.0,4.0,0.0,95.0,1.0,4.0
10317.000052352,011538c9fc0163c0ed541f649ff1eded7b75972a57ce2c2f955b966873383dd4220315e6c8b8a113598105156bcacaff2eec2f121d72ab723491b57f7c463151,0.0,0.0,NaN,0.0,68.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,2.0,4.0,0.0,0.0,1.0,63.0,3.0,4.0
10317.000059184,987a3c7c00ae989b13186d7aaecd01c19f0eb6ed0c3f126a0583f73e0a8176d4220278684974dbee9d50ff61d7f56661add3be690a799866b1981fcb8c0b0dfd,0.0,NaN,0.0,0.0,60.0,1.0,3.0,NaN,1.0,0.0,...,0.0,0.0,2.0,4.0,4.0,2.0,1.0,70.0,3.0,4.0
10317.000059196,7318a813ee852f81f65d1dfafe0d0b915d55c350d62b6d31b6f17aa43336a62c7e10b8d841c20c879e8284fa5beee5a95345457ac0c43ff9868d5d3c208d7c17,0.0,0.0,0.0,0.0,48.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,2.0,3.0,3.0,3.0,3.0,63.0,2.0,4.0
10317.000059951,e7ee3f1de28bdb09f18ca0972d6281980a7fdce522607cedb00982f5eb8cfe0a510af22a3dbe2dbcfeaad7e597f692e7b5525e65b5717c9372f7d1b5ce9fc068,1.0,NaN,0.0,NaN,52.0,1.0,3.0,1.0,0.0,0.0,...,0.0,0.0,1.0,4.0,0.0,0.0,1.0,62.0,3.0,3.0
10317.000059966,a46ffb32c467114056b3a9004225797bfeec984c916391e8b709b8e8b9ec6515e0548a0bc54745296b6a4556a243fedbfb74b5a0e26129e5d3bbed709b782526,0.0,0.0,0.0,NaN,42.0,1.0,2.0,0.0,1.0,0.0,...,0.0,0.0,4.0,4.0,2.0,2.0,0.0,75.0,3.0,0.0
10317.000060009,4fcef50a240ff9802cfaca3dbea1c47508e31130d254b679369e1ca46ee0fe0e4b8a0f25641675dc2c4d54ae60ece6e3e7dea56d2462a09d640e619308bdfb05,1.0,0.0,NaN,0.0,49.0,1.0,1.0,NaN,1.0,0.0,...,0.0,0.0,1.0,3.0,0.0,1.0,2.0,115.0,2.0,2.0


In [123]:
#The pure dataframe for reference
pureDF

,,acid_reflux,acne_medication,acne_medication_otc,add_adhd,age_corrected,alcohol_consumption,alcohol_frequency,alcohol_types_beercider,alcohol_types_red_wine,alcohol_types_sour_beers,...,thyroid,tonsils_removed,types_of_plants,vegetable_frequency,vitamin_b_supplement_frequency,vitamin_d_supplement_frequency,vivid_dreams,weight_kg,whole_eggs,whole_grain_frequency
sample_name,host_subject_id,,,,,,,,,,,,,,,,,,,,,
10317.000039633,65547100856692be9ef5c27b03fb92409eaf98e317f84bbf69f7b3db4a65e7c14da32bb4cdba1e56c67af4abd96f7c0a5799a20b05b59838c1a5dd060d24c2fa,0.0,0.0,0.0,0.0,58.0,1.0,3.0,1.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,72.0,0.0,4.0
10317.000039731,2983e6ca9e4039d71c48585b8cc237f5f5143944260cb5d915a882922e450dd111971305392dfaf8c20bf75094ef4e70665ef37a2f7d44d8aa37b5e4ea6c4d11,1.0,0.0,0.0,1.0,64.0,1.0,1.0,1.0,1.0,0.0,...,0.0,1.0,1.0,3.0,4.0,4.0,1.0,70.0,3.0,3.0
10317.000039866,7e4ad3f6d351b74239d2e9388b56ec309db43f676ca8a7569d4b5dfc3278b9584ecd224f4f276b76d2efcee2052d2c3e6dc85e2655aa607232fda53b1e93de36,0.0,0.0,0.0,0.0,55.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,4.0,1.0,4.0,0.0,95.0,1.0,4.0
10317.000052352,011538c9fc0163c0ed541f649ff1eded7b75972a57ce2c2f955b966873383dd4220315e6c8b8a113598105156bcacaff2eec2f121d72ab723491b57f7c463151,0.0,0.0,0.0,0.0,68.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,2.0,4.0,0.0,0.0,1.0,63.0,3.0,4.0
10317.000059184,987a3c7c00ae989b13186d7aaecd01c19f0eb6ed0c3f126a0583f73e0a8176d4220278684974dbee9d50ff61d7f56661add3be690a799866b1981fcb8c0b0dfd,0.0,0.0,0.0,0.0,60.0,1.0,3.0,0.0,1.0,0.0,...,0.0,0.0,2.0,4.0,4.0,2.0,1.0,70.0,3.0,4.0
10317.000059196,7318a813ee852f81f65d1dfafe0d0b915d55c350d62b6d31b6f17aa43336a62c7e10b8d841c20c879e8284fa5beee5a95345457ac0c43ff9868d5d3c208d7c17,0.0,0.0,0.0,0.0,48.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,2.0,3.0,3.0,3.0,3.0,63.0,2.0,4.0
10317.000059951,e7ee3f1de28bdb09f18ca0972d6281980a7fdce522607cedb00982f5eb8cfe0a510af22a3dbe2dbcfeaad7e597f692e7b5525e65b5717c9372f7d1b5ce9fc068,1.0,0.0,0.0,0.0,52.0,1.0,3.0,1.0,0.0,0.0,...,0.0,0.0,1.0,4.0,0.0,0.0,1.0,62.0,3.0,3.0
10317.000059966,a46ffb32c467114056b3a9004225797bfeec984c916391e8b709b8e8b9ec6515e0548a0bc54745296b6a4556a243fedbfb74b5a0e26129e5d3bbed709b782526,0.0,0.0,0.0,0.0,42.0,1.0,2.0,0.0,1.0,0.0,...,0.0,0.0,4.0,4.0,2.0,2.0,0.0,75.0,3.0,0.0
10317.000060009,4fcef50a240ff9802cfaca3dbea1c47508e31130d254b679369e1ca46ee0fe0e4b8a0f25641675dc2c4d54ae60ece6e3e7dea56d2462a09d640e619308bdfb05,1.0,0.0,0.0,0.0,49.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,1.0,3.0,0.0,1.0,2.0,115.0,2.0,2.0


In [160]:
FixedfalseMatMask=np.zeros((47,pureDF.shape[1]), dtype=bool).transpose()
flipcount=0
for ar in missingmask:
    for i in ar:
        FixedfalseMatMask[flipcount][i]=True
    flipcount=flipcount+1
FixedfalseMatMask=FixedfalseMatMask.transpose()
FixedfalseMatMask

array([[ True, False, False, ..., False,  True,  True],
       [False, False,  True, ..., False, False, False],
       [ True, False, False, ..., False,  True, False],
       ...,
       [False, False, False, ..., False, False,  True],
       [ True, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [161]:
#Lets run the iterative imputator
from fancyimpute import IterativeImputer
print(nanifiedpureDF.shape)
X_filled_ii = IterativeImputer().fit_transform(nanifiedpureDF)
print(X_filled_ii.shape)
ii_mse = ((X_filled_ii[FixedfalseMatMask] - pureDF.values[FixedfalseMatMask]) ** 2).mean()
print("Iterative Imputer norm minimization MSE: %f" % ii_mse)

(47, 139)


C:\Users\Ryan\Anaconda2\envs\py3\lib\site-packages\sklearn\linear_model\ridge.py:957: RuntimeWarning: invalid value encountered in true_divide
  return (c / G_diag) ** 2, c
C:\Users\Ryan\Anaconda2\envs\py3\lib\site-packages\sklearn\linear_model\ridge.py:957: RuntimeWarning: invalid value encountered in true_divide
  return (c / G_diag) ** 2, c
C:\Users\Ryan\Anaconda2\envs\py3\lib\site-packages\sklearn\linear_model\ridge.py:957: RuntimeWarning: invalid value encountered in true_divide
  return (c / G_diag) ** 2, c
C:\Users\Ryan\Anaconda2\envs\py3\lib\site-packages\sklearn\linear_model\ridge.py:957: RuntimeWarning: invalid value encountered in true_divide
  return (c / G_diag) ** 2, c
C:\Users\Ryan\Anaconda2\envs\py3\lib\site-packages\sklearn\linear_model\ridge.py:957: RuntimeWarning: invalid value encountered in true_divide
  return (c / G_diag) ** 2, c
C:\Users\Ryan\Anaconda2\envs\py3\lib\site-packages\sklearn\linear_model\ridge.py:957: RuntimeWarning: invalid value encountered in true

(47, 137)


C:\Users\Ryan\Anaconda2\envs\py3\lib\site-packages\sklearn\linear_model\ridge.py:957: RuntimeWarning: invalid value encountered in true_divide
  return (c / G_diag) ** 2, c
C:\Users\Ryan\Anaconda2\envs\py3\lib\site-packages\sklearn\linear_model\ridge.py:957: RuntimeWarning: invalid value encountered in true_divide
  return (c / G_diag) ** 2, c
C:\Users\Ryan\Anaconda2\envs\py3\lib\site-packages\sklearn\linear_model\ridge.py:957: RuntimeWarning: invalid value encountered in true_divide
  return (c / G_diag) ** 2, c
C:\Users\Ryan\Anaconda2\envs\py3\lib\site-packages\sklearn\linear_model\ridge.py:957: RuntimeWarning: invalid value encountered in true_divide
  return (c / G_diag) ** 2, c


IndexError: boolean index did not match indexed array along dimension 1; dimension is 137 but corresponding boolean dimension is 139